In [1]:
import pandas as pd
import evalml
import woodwork as ww
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from featuretools.selection import remove_low_information_features, remove_highly_null_features, remove_single_value_features, remove_highly_correlated_features

In [14]:
df = pd.read_csv('merged.csv')
df = df.drop(['Unnamed: 0'], axis = 1)
df.head()

,country,isonum,isocode,year,rank_educ,educ,qslnl,secvocp,teryrs,edqualsc,...,envi,fwateruse,airpollution,wasterwater,fishstock,marineprotect,improved_drinkwater,pestreg,terrestprotect,prosperity_score
0,Afghanistan,4,AFG,2007,143,22.494400,-7.178585,0.714436,0.207426,337.589355,...,39.293343,43.011665,0.1400,0.000000,35.323078,13.537368,42.6,0,0.366882,37.658975
1,Angola,24,AGO,2007,125,35.916668,-7.178585,39.072861,0.099019,280.802673,...,47.982162,0.476892,0.0250,0.000000,4.500000,0.066623,46.4,3,12.399823,43.880780
2,Albania,8,ALB,2007,72,56.318413,-7.178585,7.345947,0.127648,425.420013,...,51.049442,4.873606,0.2750,5.136225,37.099178,1.624084,95.9,5,9.804400,54.893284
3,United Arab Emirates,784,ARE,2007,52,59.942955,-7.178585,0.494192,0.488107,480.829529,...,62.051479,100.000000,0.1975,61.308900,30.838837,2.571971,99.6,23,5.616998,61.111192
4,Argentina,32,ARG,2007,92,51.995552,-7.178585,6.997118,0.265608,421.549591,...,64.909729,12.938356,0.0050,18.062500,12.280000,1.103726,97.7,23,5.466403,58.557573


In [27]:
df.groupby('country')['gdp5'].min()

country
Afghanistan    0.040925
Albania        0.030713
Algeria        0.005329
Angola         0.012823
Argentina      0.029559
                 ...   
Venezuela     -0.004891
Vietnam        0.045396
Yemen         -0.028145
Zambia         0.037333
Zimbabwe      -0.022775
Name: gdp5, Length: 149, dtype: float64

In [56]:
filtered = df[df['country'].isin(['Chad', 'Togo', 'Zimbabwe', 'Ivory Coast', 'Georgia'])]
filtered[['country', 'year', 'prosperity_score']]

,country,year,prosperity_score
26,Ivory Coast,2007,44.145295
49,Georgia,2007,51.980642
131,Chad,2007,34.067548
132,Togo,2007,40.598943
148,Zimbabwe,2007,43.235151
175,Ivory Coast,2008,44.288615
198,Georgia,2008,53.208659
280,Chad,2008,36.354614
281,Togo,2008,40.428445
297,Zimbabwe,2008,41.921147


In [49]:
filtered.corr()['prosperity_score'].sort_values(ascending=False).head(10)

prosperity_score       1.000000
heal                   0.938947
stardtf                0.916997
improved_drinkwater    0.901337
econ                   0.880024
gove                   0.874449
busi                   0.871184
educ                   0.869613
imeas                  0.869306
geff                   0.864689
Name: prosperity_score, dtype: float64

In [57]:
filtered = df[df['country'].isin(['Chad', 'Togo', 'Zimbabwe', 'Ivory Coast', 'Georgia'])]
selected_df = df[['country', 'year', 'heal', 'stardtf', 'improved_drinkwater', 'econ', 'gove']]
selected_df
#selected_df.groupby('year')['heal'].min()

,country,year,heal,stardtf,improved_drinkwater,econ,gove
0,Afghanistan,2007,51.600807,82.87,42.6,53.251713,29.760307
1,Angola,2007,57.914005,34.94,46.4,51.743580,30.910105
2,Albania,2007,74.527748,68.21,95.9,57.979511,42.531654
3,United Arab Emirates,2007,77.575127,71.61,99.6,67.167816,48.831993
4,Argentina,2007,74.439102,72.77,97.7,62.476505,45.032742
...,...,...,...,...,...,...,...
1187,Vietnam,2014,73.153931,77.68,96.4,68.756935,40.724960
1188,Yemen,2014,59.361298,74.43,54.9,41.749683,28.166433
1189,South Africa,2014,58.200974,81.18,92.8,55.837296,61.965115
1190,Zambia,2014,59.711163,86.91,64.6,47.259438,47.728058


In [58]:
min_df = gdp_df.groupby('country')['gdp5'].min()

country
Chad           0.021602
Georgia        0.049308
Ivory Coast   -0.015723
Togo           0.000646
Zimbabwe      -0.022775
Name: gdp5, dtype: float64

In [30]:
max_df = gdp_df.groupby('country')['gdp5'].max()

In [33]:
max_df - min_df

country
Chad           0.020694
Georgia        0.018673
Ivory Coast    0.037285
Togo           0.022426
Zimbabwe       0.094416
Name: gdp5, dtype: float64